In [2]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC


In [ ]:
def Learn_Multiclass_SVM_Model(data,labels):
    model_init = OneVsRestClassifier(SVC(kernel="poly"))
    
    C_exps = [-2,-1,0,1,2,3,4,5,6,7]
    gamma_exps = [-7,-6,-5,-4,-3,-2]
    C = [2**exp for exp in C_exps]
    gamma = [2**exp for exp in gamma_exps]
    
    i_C = 0
    i_g = 0
    for C_i in C:
        i_g = 0
        for gamma_i in gamma:
            # shrinking? probability? tol?
            clf = SVC(C=C_i, kernel='rbf', degree=3, gamma=gamma_i, shrinking=True, probability=False, tol=0.001, cache_size=200, verbose=False)
            scores = cross_val_score(clf, X_train, X_test, cv=5)
            print("For C=2^%d, gamma=2^%d ; Accuracy: %0.2f (+/- %0.2f)" % (C_exps[i_C],gamma_exps[i_g],scores.mean(), scores.std() * 2))
            i_g +=1
        i_C += 1
    
    return model

def Predict(model, test_data):
    predictions = model.predict(test_data)
    return predictions

